In [2]:
import os 
from pathlib import Path

In [3]:
%pwd

'c:\\code\\ML\\e-commerce\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\code\\ML\\e-commerce'

In [6]:
#config.yaml ----> done

In [7]:
#entity
from dataclasses import dataclass

@dataclass
class DataIngestionconfig:
  root_dir : Path
  source_url : str
  local_data_file : Path

 

In [8]:
#configuration

In [22]:
from e_commerce.utils.common import read_yaml , create_directories 
from e_commerce.constants import *
from e_commerce.entity.config_entity import DataIngestionconfig

In [10]:
class ConfigurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        data_ingestion_config = DataIngestionconfig(root_dir=config.root_dir,
                                                    source_url=config.source_url,
                                                    local_data_file=config.local_data_file)
        return data_ingestion_config
    


In [11]:
#components

In [26]:
import os
import urllib.request 
import zipfile
from e_commerce import logger
from e_commerce.utils.common import get_size
from e_commerce.entity.config_entity import DataIngestionconfig
import requests

class DataIngestionComponent:
    def __init__(self,config:DataIngestionconfig):
        self.config = config

    def download_file(self):
        url=self.config.source_url
        output_path=self.config.local_data_file
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        logger.info(f"Starting download from {self.config.source_url}...")
        try:
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(output_path, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)
            print(f"File successfully downloaded to: {output_path}")
        except Exception as e:
            print(f"Download failed: {e}")
        return self.config.local_data_file


        

In [13]:
#pipeline

In [27]:
try : 
    config  = ConfigurationManager()
    data_ingestion_configuration = config.get_data_ingestion_config()
    data_ing_comp = DataIngestionComponent(config=data_ingestion_configuration)
    data_ing_comp.download_file()
except Exception as e:
    raise e

[2025-05-25 22:33:39,247: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-25 22:33:39,349: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-25 22:33:39,349: INFO: common: created directory at: artifacts]
[2025-05-25 22:33:39,390: INFO: 3673157110: Starting download from https://raw.githubusercontent.com/kartheek2003/data_zip_repo/main/online_retail.xlsx...]
File successfully downloaded to: artifacts/data_ingestion/online_retail.xlsx
